In [ ]:
import fiftyone as fo

# Load COCO dataset
dataset = fo.zoo.load_zoo_dataset("coco-2017", split="validation", max_samples=100)

# Launch the app for interactive visualization
session = fo.launch_app(dataset)

In [11]:
import pickle
from pathlib import Path

# Use a raw string for windows path, and open the file in binary mode
pkl_path = r"C:\Users\john\Documents\programming_dirty\me744_project\datasets\visualizations\Fuji-Apple-Segmentation-Augmented\pred_7.pkl"

with open(pkl_path, "rb") as f:
    data = pickle.load(f)

# Optionally, display the loaded data type or keys for inspection
print(type(data))
if isinstance(data, dict):
    print(data.keys())

print(data["boxes"].device)

<class 'dict'>
dict_keys(['boxes', 'labels', 'scores', 'masks'])
cuda:0


In [1]:
import fiftyone as fo

# data_path="Car-Parts-Segmentation/trainingset/JPEGImages"
# labels_path="Car-Parts-Segmentation/trainingset/annotations.json"

data_path = "../datasets/Fuji-Apple-Segmentation_coco/trainingset/JPEGImages"
labels_path = "../datasets/Fuji-Apple-Segmentation_coco/trainingset/annotations.json"

data_path = "../datasets/image_envy_5000_coco/trainingset/JPEGImages"
labels_path = "../datasets/image_envy_5000_coco/trainingset/annotations.json"

data_path = "../datasets/Fuji-Apple-Segmentation_with_envy_mask_coco/trainingset/JPEGImages"
labels_path = "../datasets/Fuji-Apple-Segmentation_with_envy_mask_coco/trainingset/annotations.json"

coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path,
    include_id=True,
)

# COCO categories are also imported
print(coco_dataset.info["categories"])
# [{'id': 1, 'name': 'airplane', 'supercategory': None}, ...]

# print(coco_dataset)
session = fo.launch_app(coco_dataset)

c:\Users\john\Documents\programming_dirty\me744_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
 100% |█████████████████| 399/399 [12.9s elapsed, 0s remaining, 31.9 samples/s]      
[{'id': 1, 'name': 'apple', 'supercategory': 'fruit'}, {'id': 2, 'name': 'branches', 'supercategory': 'tree'}]


In [ ]:
from core import CocoSegmentationDataset, build_model, detection_collate
import matplotlib.pyplot as plt

build_model(num_classes=9)

# Play with COCO dataset
train_ds = CocoSegmentationDataset(
    "Fuji-Apple-Segmentation/trainingset/JPEGImages",
    "Fuji-Apple-Segmentation/trainingset/annotations.json",
)

train_ds = CocoSegmentationDataset(
    "image_envy_5000_coco/trainingset/JPEGImages",
    "image_envy_5000_coco/trainingset/annotations.json",
)  # train_ds = CocoSegmentationDataset(
#     "Car-Parts-Segmentation/trainingset/JPEGImages",
#     "Car-Parts-Segmentation/trainingset/annotations.json",
# )
# Print shape of all masks in the dataset
# for idx in range(len(train_ds)):
#     _, target = train_ds[idx]
#     if "masks" in target:
#         print(f"Sample {idx}: mask shape = {target['masks'].shape}")
#     else:
#         print(f"Sample {idx}: No mask found")


def display_ds_item(train_ds, idx):
    # __getitem__ usually returns (image, target)
    img, target = train_ds[i]
    img_np = img.permute(1, 2, 0).cpu().numpy()
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(img_np)
    axes[0].set_title("Image")
    axes[0].axis("off")
    # Visualize mask (showing the merged mask from all instances)
    if "masks" in target and target["masks"].ndim == 3:
        mask_all = target["masks"].sum(dim=0)
        axes[1].imshow(mask_all.cpu().numpy(), cmap="gray")
        axes[1].set_title("Combined Mask")
    else:
        axes[1].text(0.5, 0.5, "No mask", ha="center", va="center")
    axes[1].axis("off")
    plt.tight_layout()
    plt.show()


img_keys = list(train_ds.coco.imgs.keys())
for idx in img_keys:
    ann_ids = train_ds.coco.getAnnIds(imgIds=idx)
    print("ann_ids:", ann_ids)
    print(f"train_ds.coco.loadImgs({idx}):")
    print(train_ds.coco.loadImgs(idx))
    anns = train_ds.coco.loadAnns(ann_ids)
    # print("anns:")
    # print(anns)
    for ann in anns:
        print("train_ds.coco.annToMask(ann):")
        mask = train_ds.coco.annToMask(ann)
        print("mask.shape:", mask.shape)
        print("mask.sum():", mask.sum())

# Visualize images and masks: show first 5 samples
for i in range(15):
    display_ds_item(train_ds, i)

In [ ]:
sample = coco_dataset.first()
print(sample)
import json

print(json.dumps(sample.to_dict(), indent=2))

# Get the absolute path of the current file

with open("filename", "w") as f:
    json.dump(sample.to_dict(), f)

In [16]:
from pathlib import Path

files = [
    x.name
    for x in Path(
        r"C:\Users\john\Documents\programming_dirty\me744_project\datasets\Fuji-Apple-Segmentation_with_envy_mask_coco\trainingset\JPEGImages"
    ).iterdir()
    if x.is_file()
]
with open("output.txt", "w") as f:
    f.write("\n".join(files))

In [17]:
import numpy as np
import torch
import scipy.ndimage
from PIL import Image

APPLE_MASK_COLOR = [216, 147, 236]


def construct_ground_truth(image_np, mask_color=APPLE_MASK_COLOR):
    """
    Construct ground truth dict from an image and a specific color mask.
    Args:
        image_np: Label image as numpy array [H, W, C] (RGB or RGBA)
        mask_color: List/Tuple of [R, G, B] to match
    Returns:
        target dict with boxes, labels, masks
    """
    print(f"Constructing ground truth for {image_np.shape}")
    print(f"Mask color: {mask_color}")
    # Ensure image is numpy
    img = np.array(image_np)

    # Handle RGB vs RGBA: just check first 3 channels
    if img.shape[-1] >= 3:
        img_rgb = img[:, :, :3]
    else:
        img_rgb = img

    # Create binary mask where color matches
    # Using small tolerance for potential compression artifacts
    # mask = np.all(img_rgb == mask_color, axis=-1)
    mask = np.all(np.abs(img_rgb - mask_color) < 5, axis=-1)
    print(f"Mask: {mask}")

    if not np.any(mask):
        return {
            "boxes": torch.zeros((0, 4)),
            "labels": torch.zeros((0,), dtype=torch.int64),
            "masks": torch.zeros((0, img.shape[0], img.shape[1]), dtype=torch.uint8),
        }

    # Connected components to separate instances
    labeled_mask, num_features = scipy.ndimage.label(mask)

    boxes = []
    masks = []
    labels = []

    for i in range(1, num_features + 1):
        # boolean mask for this instance
        instance_mask = labeled_mask == i

        # Find bounding box
        rows = np.any(instance_mask, axis=1)
        cols = np.any(instance_mask, axis=0)
        if not np.any(rows) or not np.any(cols):
            continue

        y1, y2 = np.where(rows)[0][[0, -1]]
        x1, x2 = np.where(cols)[0][[0, -1]]

        # Box format: x1, y1, x2, y2
        boxes.append([x1, y1, x2 + 1, y2 + 1])
        masks.append(instance_mask)
        labels.append(1)  # Default class 1

    if not boxes:
        return {
            "boxes": torch.zeros((0, 4)),
            "labels": torch.zeros((0,), dtype=torch.int64),
            "masks": torch.zeros((0, img.shape[0], img.shape[1]), dtype=torch.uint8),
        }

    # Convert to tensors
    target = {
        "boxes": torch.as_tensor(boxes, dtype=torch.float32),
        "labels": torch.as_tensor(labels, dtype=torch.int64),
        "masks": torch.as_tensor(np.stack(masks), dtype=torch.uint8),  # [N, H, W]
    }
    print(f"Target: {target}")

    return target

img_path = "C:/Users/john/Documents/programming_dirty/me744_project/maskrcnn_coco/test_img/4997_rgb_0001_label.png"

label_img = np.array(Image.open(img_path).convert("RGB"))
target = construct_ground_truth(label_img)
print(f"Target: {target}")


Constructing ground truth for (1080, 1920, 3)
Mask color: [216, 147, 236]
Mask: [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Target: {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64), 'masks': tensor([], size=(0, 1080, 1920), dtype=torch.uint8)}
